In [ ]:
def carregar_imagem(caminho_imagem):
    with open(caminho_imagem, 'rb') as f:
        header = f.read(54)  # O cabeçalho do arquivo BMP tem 54 bytes
        largura = int.from_bytes(header[18:22], 'little')
        altura = int.from_bytes(header[22:26], 'little')
        padding = (4 - (largura * 3) % 4) % 4

        imagem = []
        for y in range(altura):
            linha = []
            for x in range(largura):
                b = int.from_bytes(f.read(1), 'little')
                g = int.from_bytes(f.read(1), 'little')
                r = int.from_bytes(f.read(1), 'little')
                linha.append([r, g, b])
            f.read(padding)  # Ignorar padding
            imagem.append(linha)
        imagem.reverse()  # A imagem é armazenada de cabeça para baixo
    return imagem

def salvar_imagem(imagem, caminho_saida):
    altura = len(imagem)
    largura = len(imagem[0])
    padding = (4 - (largura * 3) % 4) % 4

    with open(caminho_saida, 'wb') as f:
        # Escrever o cabeçalho BMP
        f.write(b'BM')
        f.write((54 + (largura * 3 + padding) * altura).to_bytes(4, 'little'))
        f.write(b'\x00\x00\x00\x00')
        f.write((54).to_bytes(4, 'little'))
        f.write((40).to_bytes(4, 'little'))
        f.write((largura).to_bytes(4, 'little'))
        f.write((altura).to_bytes(4, 'little'))
        f.write((1).to_bytes(2, 'little'))
        f.write((24).to_bytes(2, 'little'))
        f.write(b'\x00\x00\x00\x00')
        f.write((largura * altura * 3).to_bytes(4, 'little'))
        f.write(b'\x13\x0B\x00\x00')
        f.write(b'\x13\x0B\x00\x00')
        f.write(b'\x00\x00\x00\x00')
        f.write(b'\x00\x00\x00\x00')

        for y in range(altura - 1, -1, -1):
            for x in range(largura):
                f.write(bytes(imagem[y][x]))
            f.write(b'\x00' * padding)

def converter_para_cinza(imagem):
    altura = len(imagem)
    largura = len(imagem[0])
    for y in range(altura):
        for x in range(largura):
            r, g, b = imagem[y][x]
            cinza = int(0.2989 * r + 0.5870 * g + 0.1140 * b)
            imagem[y][x] = [cinza, cinza, cinza]
    return imagem

def converter_para_binarizada(imagem, limite=128):
    altura = len(imagem)
    largura = len(imagem[0])
    for y in range(altura):
        for x in range(largura):
            r, g, b = imagem[y][x]
            cinza = int(0.2989 * r + 0.5870 * g + 0.1140 * b)
            if cinza > limite:
                imagem[y][x] = [255, 255, 255]
            else:
                imagem[y][x] = [0, 0, 0]
    return imagem

# Exemplos de uso:
caminho_imagem = "/content/1-032fd6a9.bmp"
caminho_saida_cinza = '/content/imagem_cinza.bmp'
caminho_saida_binarizada = '/content/imagem_binarizada.bmp'

imagem = carregar_imagem(caminho_imagem)

# Converter para níveis de cinza
imagem_cinza = converter_para_cinza(imagem)
salvar_imagem(imagem_cinza, caminho_saida_cinza)

# Converter para binarizada
imagem_binarizada = converter_para_binarizada(imagem, limite=128)
salvar_imagem(imagem_binarizada, caminho_saida_binarizada)
